In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
engine = create_engine("sqlite:///pre_covid_copy.db")
turnstiles_df = pd.read_sql("SELECT * FROM mta_data;", engine)
turnstiles_df.head(4)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,03:00:00,REGULAR,7693605,2677444
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,07:00:00,REGULAR,7693608,2677454
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,11:00:00,REGULAR,7693627,2677547
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,15:00:00,REGULAR,7693672,2677621


### Data Cleaning
- What kind of data is given in each column?
- Do the data contain missing values? df.info()
- How many observations and columns df.info() and .shape
- Look at data types of each column df.info()
  * Dtype object means column contain strings
- Remove via strip the leading and trailing space of the columns. Sample code:
  turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
- Show column names    df.columns
- Validate that you have the dates of data you expected
- What other things need to be done? 

In [3]:
turnstiles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4145766 entries, 0 to 4145765
Data columns (total 11 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   C/A       object
 1   UNIT      object
 2   SCP       object
 3   STATION   object
 4   LINENAME  object
 5   DIVISION  object
 6   DATE      object
 7   TIME      object
 8   DESC      object
 9   ENTRIES   int64 
 10  EXITS     int64 
dtypes: int64(2), object(9)
memory usage: 347.9+ MB


In [4]:
# remove any leading and trailing space of the columns
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [5]:
# Validate Dates to analyze 1/29/2022 - 3/19/2022
turnstiles_df.DATE.value_counts().sort_index()

01/01/2020    29428
01/02/2020    30048
01/03/2020    29261
01/04/2020    29439
01/05/2020    29848
              ...  
03/27/2020    29352
12/28/2019    29437
12/29/2019    29384
12/30/2019    29607
12/31/2019    29334
Name: DATE, Length: 140, dtype: int64

### Make Time Series

In [6]:
# Turn into time series
import datetime
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, format="%m/%d/%Y %H:%M:%S")

In [7]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,03:00:00,REGULAR,7693605,2677444,2022-03-12 03:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,07:00:00,REGULAR,7693608,2677454,2022-03-12 07:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,11:00:00,REGULAR,7693627,2677547,2022-03-12 11:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,15:00:00,REGULAR,7693672,2677621,2022-03-12 15:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,19:00:00,REGULAR,7693730,2677683,2022-03-12 19:00:00


### Check for Duplicates

In [8]:
# Interlude - check for duplicates
# isolate specific turnstiles by creating boolean series masks, then using them to index into the dataframe:
mask = ((turnstiles_df["C/A"] == "A002") &
        (turnstiles_df["UNIT"] == "R051") & 
        (turnstiles_df["SCP"] == "02-00-00") & 
        (turnstiles_df["STATION"] == "59 ST"))

turnstiles_df[mask].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,03:00:00,REGULAR,7693605,2677444,2022-03-12 03:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,07:00:00,REGULAR,7693608,2677454,2022-03-12 07:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,11:00:00,REGULAR,7693627,2677547,2022-03-12 11:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,15:00:00,REGULAR,7693672,2677621,2022-03-12 15:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2022,19:00:00,REGULAR,7693730,2677683,2022-03-12 19:00:00


In [9]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
3729424,R514,R094,00-05-00,ASTORIA BLVD,2020-01-21 11:00:00,2
3729414,R514,R094,00-05-00,ASTORIA BLVD,2020-01-19 19:00:00,2
1985392,N506,R022,00-03-02,34 ST-HERALD SQ,2022-02-05 03:00:00,2
3729404,R514,R094,00-05-00,ASTORIA BLVD,2020-01-18 03:00:00,2
2654308,R115,R029,00-00-00,PARK PLACE,2020-01-22 07:00:00,2


In [10]:
# Seems we have two entries for same time at 34 ST-HERALD SQ and FLUSHING-MAIN

# Look at ST-HERALD SQ
mask = ((turnstiles_df["C/A"] == "N506") & 
(turnstiles_df["UNIT"] == "R022") & 
(turnstiles_df["SCP"] == "00-03-02") & 
(turnstiles_df["STATION"] == "34 ST-HERALD SQ") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2022, 2, 5).date()))

turnstiles_df[mask].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
1149252,N506,R022,00-03-02,34 ST-HERALD SQ,BDFMNQRW,IND,02/05/2022,03:00:00,REGULAR,870698,1007524,2022-02-05 03:00:00
1149253,N506,R022,00-03-02,34 ST-HERALD SQ,BDFMNQRW,IND,02/05/2022,03:00:00,RECOVR AUD,870666,1007524,2022-02-05 03:00:00
1149254,N506,R022,00-03-02,34 ST-HERALD SQ,BDFMNQRW,IND,02/05/2022,07:00:00,REGULAR,870700,1007539,2022-02-05 07:00:00
1149255,N506,R022,00-03-02,34 ST-HERALD SQ,BDFMNQRW,IND,02/05/2022,11:00:00,REGULAR,870712,1007611,2022-02-05 11:00:00
1149256,N506,R022,00-03-02,34 ST-HERALD SQ,BDFMNQRW,IND,02/05/2022,15:00:00,REGULAR,870762,1007763,2022-02-05 15:00:00


In [11]:
# Look at FLUSHING-MAIN

mask2 = ((turnstiles_df["C/A"] == "R533") & 
(turnstiles_df["UNIT"] == "R055") & 
(turnstiles_df["SCP"] == "00-03-01") & 
(turnstiles_df["STATION"] == "FLUSHING-MAIN") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2022, 1, 29).date()))

turnstiles_df[mask2].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
1453861,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,03:00:00,REGULAR,5390246,3015053,2022-01-29 03:00:00
1453862,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,07:00:00,REGULAR,5390298,3015082,2022-01-29 07:00:00
1453863,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,11:00:00,REGULAR,5390516,3015160,2022-01-29 11:00:00
1453864,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,15:00:00,REGULAR,5390653,3015270,2022-01-29 15:00:00
1453865,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,15:00:00,RECOVR AUD,5390653,3015269,2022-01-29 15:00:00


### Remove Duplicates
Dupe data wouldn't seem to shift results. Best to Remove duplicates so there's no confusion. 

In [12]:
# need to create a subset. If DESC was included it wouldn't find dupe because of unique DESC for dupe. 
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [13]:
# check to see if dupes were removed - yes they are
mask2 = ((turnstiles_df["C/A"] == "R533") & 
(turnstiles_df["UNIT"] == "R055") & 
(turnstiles_df["SCP"] == "00-03-01") & 
(turnstiles_df["STATION"] == "FLUSHING-MAIN") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2022, 1, 29).date()))

turnstiles_df[mask2].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
1453861,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,03:00:00,REGULAR,5390246,3015053,2022-01-29 03:00:00
1453862,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,07:00:00,REGULAR,5390298,3015082,2022-01-29 07:00:00
1453863,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,11:00:00,REGULAR,5390516,3015160,2022-01-29 11:00:00
1453864,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,15:00:00,REGULAR,5390653,3015270,2022-01-29 15:00:00
1453866,R533,R055,00-03-01,FLUSHING-MAIN,7,IRT,01/29/2022,19:00:00,REGULAR,5390768,3015443,2022-01-29 19:00:00


In [18]:
# export dataframe to csv
#turnstiles_df.to_csv('pre_covid_turnstiles_cleaned.csv')

In [15]:
# import pickle

In [16]:
# turnstiles_df.to_pickle("turnstiles_cleaned.pkl")

In [17]:
# export dataframe to csv turn off index
#turnstiles_df.to_csv('turnstiles_cleaned2.csv', index=False)